In [ ]:
# %pip install -qU pypdf langchain_community

#In this section I used the anthropic API Key

In [6]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

106


In [7]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

FORM 10-K FORM 10-K
{'source': '414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf', 'page': 0}


In [31]:
url = "https://www.forex.com/en/news-and-analysis/gbpusd-eurusd-forecast-two-trades-to-watch-2024-11-7/#:~:text=EUR%2FUSD%20forecast%20%E2%80%93%20technical%20analysis&text=The%20price%20fell%20to%20a,SMA%20into%20play%20at%201.0870."
url


'https://www.forex.com/en/news-and-analysis/gbpusd-eurusd-forecast-two-trades-to-watch-2024-11-7/#:~:text=EUR%2FUSD%20forecast%20%E2%80%93%20technical%20analysis&text=The%20price%20fell%20to%20a,SMA%20into%20play%20at%201.0870.'

In [51]:
from langchain_unstructured import UnstructuredLoader
url="https://www.actionforex.com/category/technical-outlook/eurusd-outlook/"
loader = UnstructuredLoader(web_url=url)
docs = loader.load()

for doc in docs:
    print(f"{doc}\n")

page_content='Facebook Mail RSS Twitter' metadata={'category_depth': 0, 'link_texts': ['Facebook', 'Mail', 'RSS', 'Twitter'], 'link_urls': ['https://www.facebook.com/Action4X/', '/general/action-insight-forex-newsletters/', '/general/forex-rss-feeds/', 'https://twitter.com/actionforexco'], 'languages': ['eng'], 'filetype': 'text/html', 'url': 'https://www.actionforex.com/category/technical-outlook/eurusd-outlook/', 'category': 'Title', 'element_id': '2b170d9577c01377008e8a3e35fcfbb7'}

page_content='Markets' metadata={'category_depth': 1, 'link_texts': ['Markets'], 'link_urls': ['#'], 'languages': ['eng'], 'filetype': 'text/html', 'parent_id': '2b170d9577c01377008e8a3e35fcfbb7', 'url': 'https://www.actionforex.com/category/technical-outlook/eurusd-outlook/', 'category': 'ListItem', 'element_id': '04f845a55cc57f5ba0bccfa4db6800cb'}

page_content='Live Comments' metadata={'category_depth': 2, 'link_texts': ['Live Comments'], 'link_urls': ['https://www.actionforex.com/category/live-commen

In [34]:
!pip install pdfkit

You should consider upgrading via the 'C:\Users\Ricmwas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
# This article 
from langchain_unstructured import UnstructuredLoader

page_url = "https://www.actionforex.com/category/technical-outlook/eurusd-outlook/"
loader = UnstructuredLoader(web_url=page_url)

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

In [57]:
docs

[Document(metadata={'category_depth': 0, 'link_texts': ['Facebook', 'Mail', 'RSS', 'Twitter'], 'link_urls': ['https://www.facebook.com/Action4X/', '/general/action-insight-forex-newsletters/', '/general/forex-rss-feeds/', 'https://twitter.com/actionforexco'], 'languages': ['eng'], 'filetype': 'text/html', 'url': 'https://www.actionforex.com/category/technical-outlook/eurusd-outlook/', 'category': 'Title', 'element_id': '2b170d9577c01377008e8a3e35fcfbb7'}, page_content='Facebook Mail RSS Twitter'),
 Document(metadata={'category_depth': 1, 'link_texts': ['Markets'], 'link_urls': ['#'], 'languages': ['eng'], 'filetype': 'text/html', 'parent_id': '2b170d9577c01377008e8a3e35fcfbb7', 'url': 'https://www.actionforex.com/category/technical-outlook/eurusd-outlook/', 'category': 'ListItem', 'element_id': '04f845a55cc57f5ba0bccfa4db6800cb'}, page_content='Markets'),
 Document(metadata={'category_depth': 2, 'link_texts': ['Live Comments'], 'link_urls': ['https://www.actionforex.com/category/live-c

In [ ]:
import getpass
import os
#in the open api key use an api key from openapi

api_key= ""

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", api_key=api_key)

In [18]:
#%pip install langchain_openai
# %pip install llama-index-embeddings-voyageai
# !pip install llama-index

In [5]:
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_anthropic import AnthropicLLM
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

#in the open api key use an api key from openapi

openai_api=""

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings(api_key=openai_api)
)

retriever = vectorstore.as_retriever()

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a financial assistant focused on analyzing market trends and giving trading recommendations. "
    "Use the following information from the article to provide a clear trading direction based on the "
    "market analysis given. If the article lacks clear direction or analysis, state that there is  using the current price which is {current}"
    "insufficient information. Summarize in three sentences maximum, emphasizing buy, sell, or hold actions "
    "and briefly noting key technical indicators or trends that support the recommendation."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What trading action (buy, sell, hold) is suggested based on the article, and what key indicators support this decision?"})

results

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


{'input': 'What trading action (buy, sell, hold) is suggested based on the article, and what key indicators support this decision?',
 'context': [Document(id='52c06675-c7ad-404e-93ce-9c6a728a63ae', metadata={'category_depth': 3, 'link_texts': ['Trading Strategies'], 'link_urls': ['https://www.actionforex.com/category/articles/trading-strategies/'], 'languages': ['eng'], 'filetype': 'text/html', 'parent_id': 'bcb4c817ec985d6219f006807b211d9d', 'url': 'https://www.actionforex.com/category/technical-outlook/eurusd-outlook/', 'category': 'ListItem', 'element_id': '3a3f882511635c0d37ec8b8a9ef4ac69'}, page_content='Trading Strategies'),
  Document(id='4393984a-dcd9-4deb-8dad-edbab70e99d7', metadata={'category_depth': 3, 'link_texts': ['Trading Strategies'], 'link_urls': ['https://www.actionforex.com/category/articles/trading-strategies/'], 'languages': ['eng'], 'filetype': 'text/html', 'parent_id': '895b7890b5379b204e5b087d99bc8252', 'url': 'https://www.actionforex.com/category/technical-out

In [61]:
results['answer']

"The article does not provide sufficient information to make a specific trading recommendation. There is no clear analysis of market trends, technical indicators, or trading strategies presented. Without more details about price movements, chart patterns, or other relevant market data, it's not possible to suggest a definitive buy, sell, or hold action."

In [17]:
# Install package, compatible with API partitioning
#%pip install --upgrade --quiet langchain-unstructured unstructured-client unstructured "unstructured[pdf]" python-magic

page_content='Example Domain' metadata={'category_depth': 0, 'languages': ['eng'], 'filetype': 'text/html', 'url': 'https://www.example.com', 'category': 'Title', 'element_id': 'fdaa78d856f9d143aeeed85bf23f58f8'}

page_content='This domain is for use in illustrative examples in documents. You may use this domain in literature without prior coordination or asking for permission.' metadata={'languages': ['eng'], 'filetype': 'text/html', 'parent_id': 'fdaa78d856f9d143aeeed85bf23f58f8', 'url': 'https://www.example.com', 'category': 'NarrativeText', 'element_id': '3652b8458b0688639f973fe36253c992'}

page_content='More information...' metadata={'category_depth': 0, 'link_texts': ['More information...'], 'link_urls': ['https://www.iana.org/domains/example'], 'languages': ['eng'], 'filetype': 'text/html', 'url': 'https://www.example.com', 'category': 'Title', 'element_id': '793ab98565d6f6d6f3a6d614e3ace2a9'}



# The tasks
1. Look for the best fx trading news articles. The artcles should provide information on which direction to trade. Be keen on the date. 
2. Improve the system promte so that the model give information on trading. 